<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/Wildfire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Alison Soong


# Notes

Risk factors:
- High temperatures (extreme heat)
- Low humidity

Algorithm ideas:
- https://www.ci.richmond.ca.us/DocumentCenter/View/53610/City-of-Richmond-Wildfire-Preparedness-and-Evacuation-Guide "A home within 10 miles of a wildfire will most likely be effected by wind driven embers which can be a risk to your property" --> might be a good buffer area to look at


Datasets:
- https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#bands
- https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD64A1


# Import libraries

In [1]:
import ee # Import Earth Engine API

In [2]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jSPICRHXiA-0_q01y2SD1c073AgOqWABxw8es0CzzmU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgcIKIL2M373wP6WOsJEEFg4vR_tMcKJNdYw-X3_GC0S0VvtSw0Q2A

Successfully saved authorization token.


In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [4]:
try:
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap seaborn matplotlib
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
        !conda install seaborn matplotlib -y
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt

# Other ideas

In [ ]:
# Map.addLayer(population2, pop_params, "Population 2020 masked")
# dates = np.arange(2000, 2020)
# for year in dates:
#     date1 = str(year)+'-01-01'
#     date2 = str(year+1)+'-01-01'
#     burned = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))
#     popdataset = ee.ImageCollection("WorldPop/GP/100m/pop")\
#       .filterDate(date1, date2)
#     population = popdataset.mean().clip(california_area)
#     burnedImage = burned.select('BurnDate').mean().clip(california_area)
#     title = "population " + str(year)
#     title2 = "burned " + str(year)
#     Map.addLayer(population, pop_params, title, False)
#     Map.addLayer(burnedImage, burned_params2, title2, False, 0.3)

# burnedBuffer = burned.reduceToVectors().reproject(proj);

# burnedBuffer.getInfo()

# population2 = popdataset2020.mean().clip(burnedBuffer)

# buffer around areas 
# classes = burned.reduceToVectors(
#   reducer=ee.Reducer.countEvery(), 
#   geometry=california_area,
#   scale= 30,
#   maxPixels=1e9
# )

# result = ee.FeatureCollection(classes);

# burnedBuffer = burned2020.select('BurnDate').mean().clip(california_area).geometry()


# Looking at populations and burn scar (burn areas)

In [48]:
california_area = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))

def mask(image):
  empty = 0
  qa = image.select('population');
  mask = qa.equal(0) 
  return image.updateMask(mask).select("population").copyProperties(image, ["system:time_start"])

# population grid
popdataset2020 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2020-01-01", "2021-01-01")

pop2020 = popdataset2020.mean().clip(california_area)

datamask = pop2020
qa = datamask.select('population') 
mask = qa.neq(0)
pop2020masked = pop2020.updateMask(mask.unmask()).clip(california_area)

# 2019
popdataset2019 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2001-01-01", "2002-01-01")

pop2019 = popdataset2019.mean().clip(california_area)

datamask = pop2019
qa = datamask.select('population') 
mask = qa.neq(0)
pop2019masked = pop2019.updateMask(mask.unmask()).clip(california_area)

pop_params = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  # 'palette': ['black', '1fff4f', 'd4ff50']
  'palette' : ['black','bdd7e7','6baed6','2171b5']
  # 'palette': ['white', 'green', 'purple']
}

pop_params2 = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  # 'palette': ['black', '1fff4f', 'd4ff50']
  'palette' : ['eff3ff','bdd7e7','6baed6','2171b5']
  # 'palette': ['white', 'green', 'purple']
}

# burned
burnedCollection2020 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))

burned2020 = burnedCollection2020.select('BurnDate').mean().clip(california_area)

# 2019
burnedCollection2019 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2001-01-01', '2002-01-01'))

burned2019 = burnedCollection2019.select('BurnDate').mean().clip(california_area)

burned_params = {
  'min': 30.0,
  'max': 341.0,
  # 'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
  'palette':['orange', 'red']
}

roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)

Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.addLayer(pop2020, pop_params, "Population 2020", True)
Map.addLayer(pop2020masked, pop_params2, "Population 2020 masked", False)
Map.addLayer(burned2020, burned_params, 'Burned Area 2020', True, 0.6)
Map.addLayer(pop2019, pop_params, "Population 2001", True)
Map.addLayer(pop2019masked, pop_params2, "Population 2001 masked", False)
Map.addLayer(burned2019, burned_params, 'Burned Area 2001', True, 0.6)
Map.centerObject(roi, 6)
Map.addLayerControl()
Map
